# Building a Database

- Import data into SQLite
- Design a normalized database schema
- Create tables for our schema
- Insert data into our schema

We will be working with a file of Major League Baseball games from Retrosheet. Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. 

In [3]:
# Setting up the environment
import pandas as pd
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

## Importing Data

In [4]:
# Reading in the data into dataframes

game_log = pd.read_csv("game_log.csv")
park_codes = pd.read_csv("park_codes.csv")
person_codes = pd.read_csv("person_codes.csv")
team_codes = pd.read_csv("team_codes.csv")

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Previewing data
print("The shape of the dataframe is: " + str(game_log.shape))
print("\nGame Data:")
game_log.head(3)

The shape of the dataframe is: (171907, 161)

Game Data:


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [6]:
print("The shape of the dataframe is: " + str(park_codes.shape))
print("\nPark Data:")
park_codes.head(3)

The shape of the dataframe is: (252, 9)

Park Data:


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN


In [7]:
print("The shape of the dataframe is: " + str(person_codes.shape))
print("\n Person Data:")
person_codes.head(3)

The shape of the dataframe is: (20494, 7)

 Person Data:


,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN


In [8]:
print("The shape of the dataframe is: " + str(team_codes.shape))
print("\n Team Data:")
team_codes.head(3)

The shape of the dataframe is: (150, 8)

 Team Data:


,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1


## Data Preview Observations:

Game Data Associations:
- park_id in park_codes corresponds to park_id in game_log
- id in person_codes corresponds to a number of position specific columns in game_log
- team_id in team_codes corresponds to v_name and h_name (visitor and home team) in game_log

## Dataframes to SQL

In [9]:
# Creating a function to quickly run queries
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
# A function to run SQL commands
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

In [10]:
# setting up the connection
conn = sqlite3.connect("mlb.db")

In [11]:
# Propagating mlb database 
game_log.to_sql("game_log", conn, index=False)
park_codes.to_sql("park_codes", conn, index=False)
person_codes.to_sql("person_codes", conn, index=False)
team_codes.to_sql("team_codes", conn, index=False)

In [12]:
# Creating an index for the game_log table
c_game_id = "ALTER TABLE game_log ADD game_id"
c_set_game_id = '''
UPDATE game_log 
SET game_id = h_name || date || number_of_game'''

run_command(c_game_id)
run_command(c_set_game_id)

# Checking the addition of the game_id
q_check_game_id = '''
SELECT h_name, 
    date, 
    number_of_game, 
    game_id
FROM game_log
LIMIT 5'''

run_query(q_check_game_id)

,h_name,date,number_of_game,game_id
0,FW1,18710504,0,FW1187105040
1,WS3,18710505,0,WS3187105050
2,RC1,18710506,0,RC1187105060
3,CH1,18710508,0,CH1187105080
4,TRO,18710509,0,TRO187105090


## Normalization 

Removing repitition in columns
- Three columns that relate to one player, followed by three columns that relate to another player 
- Columns in game_log that have the player ID followed by the player name for each position on the team
    - This also applies to other personel positions such as umpire or team manager
    
Primary key attributes
- Player names are not attributes of the game_id but of the payer_id

Redundant data
- Park information is available in multiple tables
- League is an attribute of the team_id it appears in park_codes and game_log


## Schema

[schema]: https://s3.amazonaws.com/dq-content/193/mlb_schema.svg "MLB Database Schema"

The database [schema] optimization: 

person
- Each of the 'debut' columns have been omitted, as the data will be able to be found from other tables.
- Since the game log file has no data on coaches, we made the decision to not include this data.

park
- The start, end, and league columns contain data that is found in the main game log and can be removed.

league
- Because some of the older leagues are not well known, we will create a table to store league names.

appearance_type
- Our appearance table will include data on players with positions, umpires, managers, and awards (like winning pitcher). This table will store information on what different types of appearances are available.

## Table Normalization

In [13]:
# Creating a normalized person table
c_create_person = '''
CREATE TABLE person (
person_id TEXT PRIMARY KEY,
first_name TEXT,
last_name TEXT)'''
    
run_command(c_create_person)

In [14]:
# Populating the normalized person table
c_pop_person = '''
INSERT INTO person
SELECT 
    id,
    first,
    last
FROM person_codes'''

run_command(c_pop_person)

In [15]:
# Verifying 
q_person_check = "SELECT * FROM person LIMIT 5"
run_query(q_person_check)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [16]:
# Creating a normalized park table
c_create_park = '''
CREATE TABLE park (
park_id TEXT PRIMARY KEY,
name TEXT,
nickname TEXT,
city TEXT,
state TEXT,
notes TEXT);'''
    
run_command(c_create_park)

In [17]:
# Populating the normalized park table
c_pop_park = '''
INSERT INTO park
SELECT 
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes'''

run_command(c_pop_park)

In [18]:
# Verifying 
q_park_check = "SELECT * FROM park LIMIT 5"
run_query(q_park_check)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [19]:
# Looking at the different league codes in order to fill the league table 
park_codes['league'].value_counts()

NL    88
AL    47
AA    33
UA     9
FL     7
PL     2
Name: league, dtype: int64

In [20]:
# Creating a normalized league table
c_create_league = '''
CREATE TABLE league (
league_id TEXT PRIMARY KEY,
name TEXT);'''
    
run_command(c_create_league)

In [21]:
# Populating the normalized league table
c_pop_league = '''
INSERT INTO league
VALUES
("NL", "National League"),
("AL", "American League"),
("AA", "Double-A"),
("US", "Union Association"),
("FL", "Florida League"),
("PL", "Player's Association");
'''
run_command(c_pop_league)

In [22]:
# Verifying 
q_league_check = "SELECT * FROM league"
run_query(q_league_check)

,league_id,name
0,NL,National League
1,AL,American League
2,AA,Double-A
3,US,Union Association
4,FL,Florida League
5,PL,Player's Association


In [23]:
# Importing appearance_type table into pandas df
appearance_type_df = pd.read_csv("appearance_type.csv")

appearance_type_df.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [24]:
# Transfering table to mlb.db
appearance_type_df.to_sql("appearance_type_init", conn, index = False)

In [25]:
# Creating empty table for appearance_type with desired structure
c_create_appearance = '''
CREATE TABLE appearance_type (
appearance_type_id TEXT PRIMARY KEY,
name TEXT,
category TEXT);'''
    
run_command(c_create_appearance)

In [26]:
# Populating the normalized appearance_type table 
c_pop_appearance = '''
INSERT INTO appearance_type
SELECT *
FROM appearance_type_init'''

run_command(c_pop_appearance)

In [27]:
# Verifying 
q_appearance_check = "SELECT * FROM appearance_type LIMIT 5"
run_query(q_appearance_check)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [28]:
# Creating a normalized team table
c_create_team = '''
CREATE TABLE team (
team_id TEXT PRIMARY KEY,
league_id TEXT,
city TEXT, 
nickname TEXT,
franch_id TEXT,
FOREIGN KEY (league_id) REFERENCES league(league_id));'''
    
run_command(c_create_team)

In [29]:
# An error occured when trying to get he values from team_codes
team_codes['team_id'].value_counts()

MIL    2
SR2    1
WS6    1
MID    1
ELI    1
CAL    1
DET    1
ML2    1
IND    1
BR2    1
TL2    1
PHP    1
BRF    1
PIT    1
CLP    1
WS4    1
CL1    1
BR3    1
PTF    1
BRO    1
MIA    1
WS8    1
BAL    1
BSN    1
PTU    1
SPU    1
PHA    1
CNU    1
WS5    1
SL5    1
FLO    1
KCU    1
NY4    1
ALT    1
HR1    1
SR1    1
CL4    1
CHA    1
CIN    1
LS2    1
BLA    1
ATL    1
KCF    1
SEA    1
KCN    1
OAK    1
KEO    1
CL3    1
CHF    1
NYA    1
ARI    1
BSP    1
SE1    1
MLN    1
SFN    1
WS1    1
KCA    1
MON    1
BL2    1
DTN    1
MLA    1
BLF    1
NY2    1
WS2    1
NH1    1
LAN    1
CN2    1
SL2    1
BUF    1
KC2    1
PH3    1
NY3    1
RC1    1
PHU    1
FW1    1
MLU    1
LS3    1
IN3    1
TRO    1
ANA    1
BL1    1
WS9    1
SL3    1
SLN    1
PTP    1
RIC    1
BFN    1
BFP    1
CL2    1
BL4    1
COL    1
IN1    1
BLN    1
LAA    1
KC1    1
BR1    1
BSU    1
CN3    1
PH1    1
NYP    1
SL4    1
WSN    1
TBA    1
PH2    1
CH1    1
PRO    1
LS1    1
PHI    1
WS3    1
NYN    1
WAS    1
C

In [30]:
# There is two instances of the Brewers since they changed leagues in 1998
q_duplicate = "SELECT * FROM team_codes WHERE team_id = 'MIL'"
run_query(q_duplicate)

,team_id,league,start,end,city,nickname,franch_id,seq
0,MIL,AL,1970,1997,Milwaukee,Brewers,SE1,2
1,MIL,NL,1998,0,Milwaukee,Brewers,SE1,3


In [31]:
# Due to the teams table not perserving the league information the instance of
# the Brewers in the AL league will be dropped

c_delete_duplicate = "DELETE FROM team_codes WHERE team_id=\"MIL\" AND league=\"AL\""
run_command(c_delete_duplicate)

In [32]:
# Populating the normalized team table
c_pop_team = '''
INSERT INTO team
SELECT 
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes'''

run_command(c_pop_team)

In [33]:
# Verifying 
q_team_check = "SELECT * FROM team"
run_query(q_team_check)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


In [34]:
# Creating empty table for game with desired structure
c_create_game = '''
CREATE TABLE game (
game_id TEXT PRIMARY KEY,
date TEXT,
number_of_game INTEGER,
park_id TEXT,
length_outs INTEGER,
day TEXT,
completion TEXT,
forfeit TEXT, 
protest TEXT,
attendance INTEGER,
length_minutes INTEGER,
additional_info TEXT,
acquisition_info TEXT,
FOREIGN KEY (park_id) REFERENCES park(park_id));'''
    
run_command(c_create_game)

In [35]:
# Populating the normalized game table 
c_pop_game = '''
INSERT INTO game
SELECT game_id,
date,
number_of_game,
park_id,
length_outs,
day_of_week,
completion,
forefeit,
protest,
attendance,
length_minutes,
additional_info,
acquisition_info
FROM game_log'''

run_command(c_pop_game)

In [36]:
# Verifying 
q_game_check = "SELECT * FROM game LIMIT 5"
run_query(q_game_check)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,Thu,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,Fri,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,Sat,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,Mon,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,Tue,None,None,None,3250,145,HTBF,Y


In [37]:
q_game_log = '''SELECT sql FROM sqlite_master 
WHERE name = "game_log" AND type = "table" '''
game_log_extract = run_query(q_game_log)

In [38]:
game_log_extract

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [39]:
# Creating the team_appearance table
c_team_app_1 = """
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
);
"""

run_command(c_team_app_1)

c_team_app_2 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""

run_command(c_team_app_2)

# Verifying the data was inserted correctly 
q_team_app = """
SELECT * FROM team_appearance
WHERE game_id = (
                 SELECT MIN(game_id) from game
                )
   OR game_id = (
                 SELECT MAX(game_id) from game
                )
ORDER By game_id, home;
"""

run_query(q_team_app)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,SLU,ALT188404300,0,UA,15,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT,ALT188404300,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,BSU,WSU188409250,0,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,WSU,WSU188409250,1,UA,10,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [40]:
# Creating the person_appearance table
c_drop = "DROP TABLE IF EXISTS person_appearance"

run_command(c_drop)

c_person_app_create = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c_person_app_fill = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c_person_app_create)
run_command(c_person_app_fill)

# Iterating through the columns to create the query
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [41]:
# Verifying Data
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q_appearence = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q_appearence)

   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646114,steab101,None,WSU188409250,UHP,Home Plate,umpire
1,1646116,murnt101,BSU,WSU188409250,MM,Manager,manager
2,1646115,crane101,BSU,WSU188409250,PSP,Starting Pitcher,pitcher
3,1646118,scanm801,WSU,WSU188409250,MM,Manager,manager
4,1646117,dailh101,WSU,WSU188409250,PSP,Starting Pitcher,pitcher


In [43]:
def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,league,table
7,appearance_type_init,table
8,appearance_type,table
9,team,table


In [45]:
# Dropping tables that have not been normalized 
c_drop_game_log = "DROP TABLE IF EXISTS game_log"
c_drop_park_codes = "DROP TABLE IF EXISTS park_codes"
c_drop_team_codes = "DROP TABLE IF EXISTS team_codes"
c_drop_person_codes = "DROP TABLE IF EXISTS person_codes"

run_command(c_drop_game_log)
run_command(c_drop_park_codes)
run_command(c_drop_team_codes)
run_command(c_drop_person_codes)

show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type_init,table
4,appearance_type,table
5,team,table
6,game,table
7,team_appearance,table
8,person_appearance,table


## Project Overview

- Import CSV data into a database.
- Design a normalized schema for a large, predominantly single table data set.
- Create tables that match the schema design.
- Migrate data from unnormalized tables into our normalized tables.